<a href="https://colab.research.google.com/github/DenisFerreira/Pesquisa_Operacional/blob/main/Problema_do_menor_caminho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Otimização em Rede

# Problema do menor caminho
O problema do menor caminho é o desafio de encontrar o menor caminho que liga dois nós de uma rede conectada.

Restrito a:
*   No nó de origem só existe saída de fluxo
*   No nó de destino só existe entrada de fluxo
*   Nos demais nós o fluxo de entrada deve ser igual ao de saída e iguais a 1


> Xij = { 1  se Xij pertence ao menor caminho ou 0 se não pertence }

**Função Objetivo** será minimizar


> Min Sum Xij*Cij

**Sujeito a:** 

> Sum Xij = 1  para todo i nó de origem

> Sum Xij = -1 para todo j nó de destino

> Sum Xik - Sum Xjk == 0 para todo nó k



In [1]:
!!pip install pulp
from pulp import *

In [18]:
# Dados do problema
n_nos = [0, 1, 2, 3, 4]
no_origem =  n_nos[0]
no_destino = n_nos[1]

grafo = [
         [0,1,1,0,0],
         [0,0,1,0,0],
         [0,0,0,1,1],
         [0,1,0,0,1],
         [0,0,0,0,0]
         ]

custos = [
          [0,300,90,0,0],
          [0,0,60,0,0],
          [0,0,0,30,180],
          [0,45,0,0,150],
          [0,0,0,0,0]
]


# Variáveis de decisão
var = {}
for i in n_nos:
  for j in n_nos:
    if grafo[i][j] == 1 :
      var[(i,j)] = LpVariable(name=f'x{i}{j}', cat=LpBinary)

# Criação do Modelo
model = LpProblem("Menor_Caminho", LpMinimize)

# Função Objetiva
lista_fo = []
for key in var.keys():
    lista_fo.append( var[key] * custos[ key[0] ][ key[1] ] )
model += lpSum(lista_fo)

# Restrições

for no in n_nos:
  lista_entradas = []
  lista_saidas = []
  for aresta in var.keys():
    if aresta[0] == no:
      lista_saidas.append(var[aresta])
    elif aresta[1] == no:
      lista_entradas.append(var[aresta])
  
  # Nó de origem tem apenas uma saída
  if no == no_origem:
    model += lpSum(lista_saidas) == 1
  
  # Nó de destino tem apenas uma entrada
  elif no == no_destino:
    model += lpSum(lista_entradas) == 1

  # Todo nó que está no caminho ótimo deve ter uma entrada e uma saída
  else: 
    model += lpSum(lista_entradas) == lpSum(lista_saidas)


In [19]:
print(model)
status = model.solve()
print(LpStatus[status])
print(f'O valor ótimo é {value(model.objective)}')
for i in var.keys():
  if(value(var[i]) != 0):
    print(f'{var[i]} = {value(var[i])}')

Menor_Caminho:
MINIMIZE
300*x01 + 90*x02 + 60*x12 + 30*x23 + 180*x24 + 45*x31 + 150*x34 + 0
SUBJECT TO
_C1: x01 + x02 = 1

_C2: x01 + x31 = 1

_C3: x02 + x12 - x23 - x24 = 0

_C4: x23 - x31 - x34 = 0

_C5: x24 + x34 = 0

VARIABLES
0 <= x01 <= 1 Integer
0 <= x02 <= 1 Integer
0 <= x12 <= 1 Integer
0 <= x23 <= 1 Integer
0 <= x24 <= 1 Integer
0 <= x31 <= 1 Integer
0 <= x34 <= 1 Integer

Optimal
O valor ótimo é 165.0
x02 = 1.0
x23 = 1.0
x31 = 1.0
